<a href="https://colab.research.google.com/gist/taruma/a9dd4ea61db2526853b99600909e9c50/taruma_hk43_pivot_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Berdasarkan isu [#43](https://github.com/taruma/hidrokit/issues/43): **ask: ubah pivot table ke dataframe**.

Deskripsi permasalahan (baca isu untuk lebih merinci):

- Saya memiliki (pivot) table yang berada di dalam berkas excel.
- Saya ingin mengubah pivot table menjadi dataframe.
- (Lanjutan) Saya ingin mengambil data (pivot table) dari seluruh sheet di dalam berkas excel.
- (Lanjutan) Memeriksa apakah terdapat data yang _invalid_.

Strategi penyelesaian:

- Memperoleh pivot table dari berkas excel.
- Transformasi pivot table ke dalam bentuk biasa (kolom tunggal/tahun).
- Menggabungkan hasil transformasi untuk masing-masing stasiun.
- Saat menggabungkan diperiksa juga kondisi data (cek _invalid_).


# DATASET

In [0]:
# AKSES GOOGLE DRIVE 
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# DRIVE PATH
DRIVE_DROP_PATH = '/content/gdrive/My Drive/Colab Notebooks/_dropbox'
DRIVE_DATASET_PATH = '/content/gdrive/My Drive/Colab Notebooks/_dataset/uma_pamarayan'

In [0]:
SINGLE_DATASET = DRIVE_DATASET_PATH + '/debit_bd_pamarayan_1998_2008.xls'

# FUNGSI

In [0]:
import pandas as pd
import numpy as np
from calendar import isleap
from collections import defaultdict
from pathlib import Path

def _get_years(io):
    excel = pd.ExcelFile(io)
    years = []
    for sheet in excel.sheet_names:
        if sheet.isdigit():
            years.append(int(sheet)) 
    return sorted(years)

def _get_pivot(io, year, fmt):
    fmt_list = ['uma.debit', 'uma.hujan']

    if fmt == 'uma.debit':
        return pd.read_excel(
            io, sheet_name=str(year),
            header=None, usecols='AN:AY'
        ).iloc[16:47, :]
    
    if fmt == 'uma.hujan':
        return pd.read_excel(
            io, sheet_name=str(year),
            header=None, usecols='B:M'
        ).iloc[19:50, :]

def _get_data_oneyear(io, year, fmt):
    _drop = [59, 60, 61, 123, 185, 278, 340]
    _drop_leap = [60, 61, 123, 185, 278, 340]

    pivot_table = _get_pivot(io, str(year), fmt=fmt)
    data = pivot_table.melt().drop('variable', axis=1)
    if isleap(year):
        return data['value'].drop(_drop_leap).values
    else:
        return data['value'].drop(_drop).values    

def _get_data_allyear(io, fmt, aslist=False):
    list_years = _get_years(io)
    
    data_each_year = []

    for year in list_years:
        data = _get_data_oneyear(io, year=year, fmt=fmt)
        data_each_year.append(data)
    
    if aslist:
        return data_each_year

    return np.hstack(data_each_year)

def _get_invalid(array, check):
    dict_invalid = defaultdict(list)
    for index, element in enumerate(array):
        try:
            check(element)
            if np.isnan(check(element)):
                dict_invalid['NaN'] += [index]
        except:
            dict_invalid[element] += [index]

    return dict(dict_invalid)

def _have_invalid(array, check):
    return bool(_get_invalid(array, check=check))

def _check_invalid(array, check=np.float):
    if _have_invalid(array, check=check):
        return _get_invalid(array, check=check)
    return None

def read_folder(dataset_path, pattern, fmt, prefix='', invalid=False):
    dataset_path = Path(dataset_path)
    total_files = len(list(dataset_path.glob(pattern)))
    print('Found {} file(s)'.format(total_files))

    data_allstation = {}
    data_invalid = {}

    for counter, file in enumerate(dataset_path.glob(pattern)):
        print(':: {:^4}:\t{:s}'.format(counter+1, file.name))
        station_name = prefix + '_'.join(file.stem.split('_')[1:-2])
        data_each_station = _get_data_allyear(file, fmt=fmt)
        data_allstation[station_name] = data_each_station
        if invalid:
            data_invalid[station_name] = _check_invalid(data_each_station)

    if invalid:
        return data_allstation, data_invalid
    else:
        return data_allstation 


# DATA

## Fungsi _private_ `_get_years()`

Tujuan: Memperoleh `list` tahun di dalam berkas excel.

In [0]:
_get_years(SINGLE_DATASET)

[1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008]

## Fungsi _private_ `_get_pivot()`

Tujuan: memperoleh pivot table dari berkas excel

In [0]:
_get_pivot(SINGLE_DATASET, year=1998, fmt='uma.debit')

,39,40,41,42,43,44,45,46,47,48,49,50
16,0,0,90.12,74.6,160.28,22.1,19.96,24.82,490,56.36,158.84,54.58
17,0,0,97.9,126.44,167.16,318.52,21.46,25.22,11.36,52.38,79.96,56.45
18,0,0,88.9,128.08,132.3,105.66,21.74,139.18,54.06,19.16,58.98,26.7
19,0,0,90.3,126.72,202.8,99.38,22.5,63.68,78.68,22.46,283.04,84.02
20,0,0,210.06,74.36,80.4,173.96,21.9,173.82,14.54,135.92,166.48,81.28
21,0,0,82.9,78.08,204.54,84.58,24.94,170.98,78.28,89.18,20.9.46,82.28
22,0,0,274.42,120.18,88.12,84.06,24.5,19.46,165.48,83.58,238.98,50.18
23,0,0,216.36,75.18,77.48,24.82,23.62,19.86,168.18,80.6,81.46,166.02
24,0,0,86.84,30.38,207.28,23.26,24.02,20.82,22,20.62,79.52,166.42
25,0,0,88.68,92.98,206.4,18.66,25.82,21.42,84.04,19.52,75.92,84.38


## Fungsi _private_ `_get_data_oneyear()`

Tujuan: Memperoleh data dari _sheet_ tunggal dan disajikan dalam bentuk `array` 1D. 

In [0]:
_get_data_oneyear(SINGLE_DATASET, year=1998, fmt='uma.debit')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 90.12, 97.9, 88.9,
       90.3, 210.06, 82.9, 274.42, 216.36, 86.84, 88.68, 80.8, 84.3, 82.3,
       120.96, 123.28, 131.06, 74.86, 159.66, 69.68, 68.36, 73.56, 74.64,
       84.96, 376.54, 126.76, 128.22, 125.46, 444.62, 154.68, 11.68,
       77.12, 74.6, 126.44, 128.08, 126.72, 74.36, 78.08, 120.18, 75.18,
       30.38, 92.98, 30.38, 158.7, 348.8, 77.46, 77.32, 128.6, 161.92,
       388.18, 123.62, 129.34, 157.76, 129.98, 14.76, 84.9, 74.76, 76.88,
       124.6, 133.6, 154.96, 348.8, 160.28, 167.16, 132.3, 202.8, 80.4,
       204.54, 88.12, 77.48, 207.28, 206.4, 284.3, 351.76, 83.44, 233.52,
       81.78, 81.98, 82.18, 84.3, 86.56, 22.46, 20.74, 83.54, 15.08,
       13.92, 64.24, 19.96, 21.84, 96.9, 113.88, 85.06, 93.24, 22.1,
       318.52, 105.66, 99.38, 173.96, 84.58, 84.06, 24.82, 23.2

In [0]:
_get_data_oneyear(SINGLE_DATASET, year=1998, fmt='uma.debit').shape

(365,)

## Fungsi _private_ `_get_data_allyear()`

Tujuan: Memeroleh data dari seluruh _sheet_ dan disajikan dalam bentuk `array` 1D.

In [0]:
_get_data_allyear(SINGLE_DATASET, fmt='uma.debit')

array([0, 0, 0, ..., 20.14, 208.54, 208.14], dtype=object)

In [0]:
_get_data_allyear(SINGLE_DATASET, fmt='uma.debit').shape

(4018,)

### `aslist=True`

Tujuan: disajikan dalam bentuk `list` `array` untuk setiap tahunnya.

In [0]:
_get_data_allyear(SINGLE_DATASET, fmt='uma.debit', aslist=True)[:2]

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 90.12, 97.9, 88.9,
        90.3, 210.06, 82.9, 274.42, 216.36, 86.84, 88.68, 80.8, 84.3, 82.3,
        120.96, 123.28, 131.06, 74.86, 159.66, 69.68, 68.36, 73.56, 74.64,
        84.96, 376.54, 126.76, 128.22, 125.46, 444.62, 154.68, 11.68,
        77.12, 74.6, 126.44, 128.08, 126.72, 74.36, 78.08, 120.18, 75.18,
        30.38, 92.98, 30.38, 158.7, 348.8, 77.46, 77.32, 128.6, 161.92,
        388.18, 123.62, 129.34, 157.76, 129.98, 14.76, 84.9, 74.76, 76.88,
        124.6, 133.6, 154.96, 348.8, 160.28, 167.16, 132.3, 202.8, 80.4,
        204.54, 88.12, 77.48, 207.28, 206.4, 284.3, 351.76, 83.44, 233.52,
        81.78, 81.98, 82.18, 84.3, 86.56, 22.46, 20.74, 83.54, 15.08,
        13.92, 64.24, 19.96, 21.84, 96.9, 113.88, 85.06, 93.24, 22.1,
        318.52, 105.66, 99.38, 173.96, 84.58, 84.0

# VERIFIKASI

## Fungsi _private_ `_have_invalid()`

Menjawab: Apakah `array` 1D ini memiliki data yang tidak bisa diubah ke dalam bentuk `check`?

In [0]:
array = _get_data_allyear(SINGLE_DATASET, fmt='uma.debit')
print(f'shape = {array.shape}; type = {array.dtype}')

shape = (4018,); type = object


In [0]:
_have_invalid(array, check=np.float)

True

## Fungsi _private_ `_get_invalid()`

Tujuan: Memperoleh nilai _invalid_ beserta indexnya dalam bentuk `dictionary`

In [0]:
_get_invalid(array, check=np.float)

{'20.9.46': [309], 'NaN': [789], 'tad': [2974]}

## Fungsi _private_ `_check_invalid()`

Tujuan: Memeriksa `array` memiliki nilai _invalid_, jika iya, apa saja?

In [0]:
_check_invalid(array, check=np.float)

{'20.9.46': [309], 'NaN': [789], 'tad': [2974]}

In [0]:
pd.DataFrame(_)

,20.9.46,NaN,tad
0,309,789,2974


# Penggunaan dalam folder

## Fungsi _public_ `read_folder()`

Tujuan: Membaca seluruh berkas excel di dalam folder yang mengikuti pola `pattern`, dan membaca isi berkas, kemudian menggabungkan seluruh hasil bacaan dalam bentuk `dictionary`.

In [0]:
data, invalid = read_folder(DRIVE_DATASET_PATH, pattern='hujan_*', fmt='uma.hujan', prefix='h_', invalid=True)

Found 8 file(s)
::  1  :	hujan_bojong_manik_1998_2008.xls
::  2  :	hujan_gunung_tunggal_1998_2008.xls
::  3  :	hujan_pasir_ona_1998_2008.xls
::  4  :	hujan_sampang_peundeuy_1998_2008.xls
::  5  :	hujan_cimarga_1998_2008.xls
::  6  :	hujan_bd_pamarayan_1998_2008.xls
::  7  :	hujan_ciminyak_cilaki_1998_2008.xls
::  8  :	hujan_gardu_tanjak_1998_2008.xls


In [0]:
pd.DataFrame(invalid).T

,-,NaN
h_bojong_manik,"[0, 1, 3, 4, 5, 6, 8, 9, 11, 14, 16, 17, 19, 2...",[2558]
h_gunung_tunggal,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16,...",NaN
h_pasir_ona,"[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 14, 17, 18...",NaN
h_sampang_peundeuy,"[2, 3, 4, 5, 16, 17, 18, 19, 20, 21, 22, 24, 2...","[0, 1]"
h_cimarga,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,...",NaN
h_bd_pamarayan,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15...","[211, 576]"
h_ciminyak_cilaki,"[0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 15, 17, ...",NaN
h_gardu_tanjak,"[0, 1, 3, 4, 5, 8, 11, 12, 17, 19, 20, 22, 45,...",NaN


In [0]:
pd.DataFrame(data, index=pd.date_range('19980101', '20081231'))

,h_bojong_manik,h_gunung_tunggal,h_pasir_ona,h_sampang_peundeuy,h_cimarga,h_bd_pamarayan,h_ciminyak_cilaki,h_gardu_tanjak
1998-01-01,-,-,-,NaN,-,-,-,-
1998-01-02,-,-,-,NaN,-,-,7,-
1998-01-03,5,-,-,-,-,64,-,5
1998-01-04,-,-,-,-,-,-,-,-
1998-01-05,-,-,-,-,-,-,-,-
1998-01-06,-,-,-,-,-,-,-,-
1998-01-07,-,-,-,5,-,-,2,9
1998-01-08,11,-,-,5,-,-,-,4
1998-01-09,-,-,2.3,1,-,-,-,-
1998-01-10,-,-,-,27,-,-,11,38


# Changelog

```
- 20190926 - 1.0.0 - Initial
```

#### Copyright &copy; 2019 [Taruma Sakti Megariansyah](https://taruma.github.io)


Source code in this notebook is licensed under a [MIT License](https://choosealicense.com/licenses/mit/). Data in this notebook is licensed under a [Creative Common Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/). 